# Unit test writing using a multi-step prompt

Complex tasks, such as writing unit tests, can benefit from multi-step prompts. In contrast to a single prompt, a multi-step prompt generates text from GPT and then feeds that output text back into subsequent prompts. This can help in cases where you want GPT to reason things out before answering, or brainstorm a plan before executing it.

In this notebook, we use a 3-step prompt to write unit tests in Python using the following steps:

1. **Explain**: Given a Python function, we ask GPT to explain what the function is doing and why.
2. **Plan**: We ask GPT to plan a set of unit tests for the function.
    - If the plan is too short, we ask GPT to elaborate with more ideas for unit tests.
3. **Execute**: Finally, we instruct GPT to write unit tests that cover the planned cases.

The code example illustrates a few embellishments on the chained, multi-step prompt:

- Conditional branching (e.g., asking for elaboration only if the first plan is too short)
- The choice of different models for different steps
- A check that re-runs the function if the output is unsatisfactory (e.g., if the output code cannot be parsed by Python's `ast` module)
- Streaming output so that you can start reading the output before it's fully generated (handy for long, multi-step outputs)

In [1]:
from plan import Planner

essay_content = """
### Essay Summary
《小马过河》讲述了一匹年轻的小马在妈妈的要求下去磨坊驮麦子的过程中，遇到了一条河。小马分别听到了牛和松鼠关于河水深浅的不同说法，犹豫不决，最后决定回家请教妈妈。经过妈妈的启发，小马明白了不能只听别人的话，要自己动脑筋，勇敢尝试。最后，小马顺利地过了河，完成了任务。这个故事教育孩子们要学会独立思考和勇敢尝试。

### Essay New words
"驮", "匹", "欣慰", "哗哗", "犹豫", "迈", "淹"
"""

student_requirement = """
### profile
我是一个小学3年级男生。
### requirements
我今天在课堂上学习过了《小马过河这篇文章》，但是我对这篇文章还没有理解，且有些生字也还不太会读写。
### prefered_format
我更喜欢通过游戏和讲故事的方式来学习。
"""

execution = Planner().plan(essay_content, student_requirement)


[system]
You are a world-class Chinese teaching and research teacher with an eagle eye for learning objectives and student's deep-level requirements. You carefully anlysis learning objectives and student's specific requirements with great detail and accuracy. You organize your analysis in markdown-formatted, bulleted lists.

[user]
Please explain learning objective and student's requirement in the following steps:
    - Analyze the essay content.
    - Explain the student's info.
    - Try to conjecture student's existing knowledge in Chinese subject.
    - Try to conjecture student's existing knowledge on given essay content.
    - Analyze the student's potential deep-level requirements as much as possible based on previous steps.
    - Define learning objectives: based on your above analysis, define clear and achievable learning objectives for this student.

    Organize your explanation in English as a markdown-formatted, bulleted list.

    # Input Parameters
    ## Essay Content


- Review the new words encountered in the essay.
- Activity:
    1. Have the student practice writing the new words and provide feedback on their handwriting. (Material type: new word handwriting rating questions)

## Integrative-Section 3: Review and Reflection
- Reflect on the learning objectives and discuss how well the student has achieved them.
- Activity:
    1. Ask the student to share their thoughts on the lesson and what they found most helpful or enjoyable.

## Conclusion
- Congratulate the student on their progress and encourage them to continue practicing their Chinese reading and writing skills.
- Discuss the importance of independent thinking and facing challenges bravely in their daily life.
[system]
You are a world-class Chinese teacher with an eagle eye for lesson plan. You write careful, accurate lesson plan. When asked to reply only with json, you write all of your code in a single block.

[user]

    # Output format
    ## lesson_plan_schema
    ```
    {
      "$sc

In [8]:
import json
execution_object = json.loads(execution)
llm_activities = [activity for section in execution_object["sections"] for activity in section["activities"] if activity["masterial_type"] == "LLM_CHAT"]
selected_activity = llm_activities[3]
print(selected_activity["description"])

Discuss the lesson the young horse learned and how it can be applied to the student's own life.


In [1]:
import gradio as gr
from tools.llm_material_bot import create_llm_material_player_bot

llm_material_player = create_llm_material_player_bot("Discuss the lesson the young horse learned and how it can be applied to the student's own life.")
llm_material_player.queue()
llm_material_player.launch()


/Users/chentienan/code/git/github/learn/ai-tutor/.venv/lib/python3.11/site-packages/gradio/utils.py:766: UserWarning: Expected 3 arguments for function <function create_llm_material_player_bot.<locals>.bot at 0x122199c60>, received 2.
  warnings.warn(
/Users/chentienan/code/git/github/learn/ai-tutor/.venv/lib/python3.11/site-packages/gradio/utils.py:770: UserWarning: Expected at least 3 arguments for function <function create_llm_material_player_bot.<locals>.bot at 0x122199c60>, received 2.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [2]:
llm_material_player.close()

Closing server running on port: 7860


Make sure to check any code before using it, as GPT makes plenty of mistakes (especially on character-based tasks like this one). For best results, use the most powerful model (GPT-4, as of May 2023).